# Hate Speech Detector - EN - Features extraction for Conv. & LSTM model

Based on [this notebook](https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb).

In [1]:
import os
import pandas as pd
import csv
import numpy as np
import pickle
from klepto.archives import dir_archive
import sys
import nltk
import string
import re
import fasttext
from pymagnitude import *

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

from tqdm.notebook import tqdm
%matplotlib inline

In [2]:
MODEL = 'lstm'

## Davidson et al. data loading

### Classes pre:
    0 - hate speech
    1 - offensive language
    2 - neither

In [3]:
if not os.path.exists('hsd/Reddit/perfect_data.pkl'):
    tweets, labels = [], []
    with open('hsd/Reddit/clean_data.csv', 'r') as f:
        for d in list(csv.reader(f))[1:]:
            tweets.append(d[1])  # tweet
            labels.append(d[2])  # class
    with open('hsd/Reddit/perfect_data.pkl', 'wb') as f:
        def chcl(c):
            return 0 if c=='2' else 1
        labels = list(map(chcl, labels))
        pickle.dump((tweets, labels), f)
else:
    with open('hsd/Reddit/perfect_data.pkl', 'rb') as f:
        tweets, labels = pickle.load(f)

### Classes post:
    0 - no hate
    1 - hate speech

In [4]:
print('Tweets: {}'.format(len(tweets)))
print('Labels: {}'.format(len(labels)))

Tweets: 55132
Labels: 55132


In [5]:
list(zip(tweets[:5], labels[:5]))

[("i joined gab to remind myself how retarded jew haters are. You wouldn't be typing on your abacus without them you retard.",
  0),
 ('This is what the left is really scared of.    https://redd.it/9rfkts', 0),
 ('That literally looks like a monkey. Why are we supposed to pretend it’s a person bc it’s wearing a red hat?',
  0),
 ('Dumb Cunt', 0),
 ('It makes you an asshole.', 0)]

## Features extraction

In [6]:
def word_tokenization(tweet):
    lemmatizer = WordNetLemmatizer() 
    tokens = word_tokenize(tweet)
    words = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    words = [lemmatizer.lemmatize(w) for w in words]
    return words

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = text_string.encode('ascii', 'ignore').decode('ascii')
    parsed_text = re.sub(space_pattern, ' ', parsed_text)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text = re.sub(hashtag_regex, '', parsed_text)
    list_words = word_tokenization(parsed_text)
    parsed_text = " ".join(list_words)
    return parsed_text

def basic_tokenize(tweet):
    tweet = " ".join(re.split(" ", tweet.lower())).strip()
    return tweet.split()

def get_pos_string(tweet):
    text = preprocess(tweet)
    tokens = basic_tokenize(text)
    tags = nltk.pos_tag(tokens)
    tag_list = [x[1] for x in tags]
    tag_str = ' '.join(tag_list)
    
    return tag_str

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

### Median sentences length

In [7]:
def median_sentences_length(data):
    all_lengths, wt_lengths, pos_lengths = [], [], []
    for d in data:
        sentence = preprocess(d)
        pos_string = get_pos_string(d)
        all_lengths.append(len(sentence.split(' ')))
        all_lengths.append(len(pos_string.split(' ')))
        wt_lengths.append(len(sentence.split(' ')))
        pos_lengths.append(len(pos_string.split(' ')))
    
    return int(np.median(all_lengths)), int(np.median(wt_lengths)), int(np.median(pos_lengths))

In [8]:
opt_length, opt_wt_length, opt_pos_length = median_sentences_length(tweets)
dim = 6*20 if MODEL == 'conv' else 200

print('Optimal all length: {}'.format(opt_length))
print('Optimal sentence length: {}'.format(opt_wt_length))
print('Optimal pos sentence length: {}'.format(opt_pos_length))

Optimal all length: 12
Optimal sentence length: 12
Optimal pos sentence length: 12


### Supervised fastText wordtokens training

In [9]:
if not os.path.exists('hsd/Reddit/fasttext.ft'):
    with open('hsd/Reddit/fasttext.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)))

# load fasttext model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_{}.bin'.format(MODEL)):
    ft_model = fasttext.load_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))
else:
    ft_model = fasttext.train_supervised('hsd/Reddit/fasttext.ft',
                                         lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_model.save_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))

### Wordtoken features

In [10]:
def get_wordtoken_fts(data, length):
    
    sentences_words = []
    for d in data:
        sentence = preprocess(d)
        sentences_words.append(sentence.split(' '))
    
    sentences_words = [pad_words(sw, length) for sw in sentences_words]
    # vecs = Magnitude(MagnitudeUtils.download_model('http://magnitude.plasticity.ai/fasttext/light/crawl-300d-2M.magnitude'))
    
    ft_matrices = []
    for sw in sentences_words:
        ft_matrix = []
        for w in sw:
            ft_matrix.append(ft_model[w])
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [11]:
wordtoken_features = get_wordtoken_fts(tweets, opt_wt_length)

In [12]:
wordtoken_features[0]

[array([-0.01975906,  0.01834656, -0.02244189, -0.00065626,  0.01436706,
        -0.00733563,  0.00707542,  0.00580501,  0.02373586,  0.04469312,
        -0.00361256,  0.00333605,  0.03175144,  0.02340886, -0.02579915,
        -0.00113104, -0.00725363,  0.01528181,  0.0085634 , -0.02039346,
        -0.0091323 ,  0.02886368,  0.01599132, -0.02732305,  0.0059767 ,
        -0.01181205,  0.02475372, -0.0176509 , -0.01138762,  0.03745642,
        -0.02825668, -0.01199644, -0.03747171, -0.0328776 , -0.03107108,
         0.00414666,  0.0239981 , -0.04372901, -0.01217095, -0.01411456,
         0.01251687,  0.03742705,  0.03041142,  0.00298579, -0.00165589,
         0.00678419, -0.00435931, -0.01013823, -0.03457943,  0.01696512,
        -0.03610675,  0.02485873, -0.04353363,  0.0230632 , -0.01902061,
        -0.02186312, -0.00674533, -0.01788346,  0.00466361,  0.02352296,
        -0.00851832,  0.03091618,  0.02614135, -0.00119754, -0.01036794,
         0.00057254, -0.03431803,  0.04573975,  0.0

### Supervised fastText pos training

In [13]:
if not os.path.exists('hsd/Reddit/fasttext_pos.ft'):
    with open('hsd/Reddit/fasttext_pos.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, get_pos_string(t)))

# load fasttext pos model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL)):
    ft_pos_model = fasttext.load_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))
else:
    ft_pos_model = fasttext.train_supervised('hsd/Reddit/fasttext_pos.ft',
                                             lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_pos_model.save_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))

### Part of speech (PoS) features

In [14]:
def get_pos_fts(data, length):

    #Get POS tags for tweets and save as a string
    pos_sentences = []
    for d in data:
        pos_string = get_pos_string(d)
        pos_sentences.append(pos_string)
        
        
    pos_tags = []
    for ps in pos_sentences:
        pos_tags.append(ps.split(' '))
    
    pos_tags = [pad_words(pt, length) for pt in pos_tags]
    
    ft_matrices = []
    for pt in pos_tags:
        ft_matrix = []
        for t in pt:
            ft_matrix.append(ft_pos_model[t])
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [15]:
pos_features = get_pos_fts(tweets, opt_pos_length)

In [16]:
pos_features[0]

[array([ 0.00902422,  0.07670008,  0.01566483, -0.14969738,  0.14069481,
        -0.1345722 ,  0.07349181, -0.0738811 ,  0.21426384,  0.08248381,
        -0.16836962,  0.13786787,  0.11268076,  0.21083143,  0.04609076,
         0.00590607,  0.11425446,  0.15854849,  0.07239335, -0.03647104,
         0.00438684,  0.11745165,  0.08598755, -0.18644038,  0.10061523,
        -0.22004129,  0.29121053, -0.31512657,  0.01587664,  0.29071498,
        -0.03230806, -0.00549846, -0.186116  , -0.43752208, -0.56268597,
        -0.02914905,  0.4930582 , -0.20828691,  0.21298778, -0.04457228,
         0.06334785,  0.09204098,  0.1885674 ,  0.01924618, -0.03847768,
         0.01178986, -0.05231132,  0.00152069, -0.07267339,  0.00932423,
        -0.1253116 ,  0.15248016, -0.18199061,  0.07463381, -0.02934593,
        -0.0155499 ,  0.04564051, -0.00201024,  0.03076228,  0.10971826,
         0.09154029,  0.15606761,  0.22033711,  0.23007193, -0.13019109,
         0.00834008, -0.10247451, -0.2431877 ,  0.1

In [17]:
np.array(wordtoken_features).shape

(55132, 12, 200)

In [18]:
np.array(pos_features).shape

(55132, 12, 200)

### All features

In [23]:
# #Now join them all up
features = np.concatenate([wordtoken_features, pos_features], axis=1)

In [24]:
features.shape

(55132, 24, 200)

In [25]:
features[0]


array([[-1.97590608e-02,  1.83465574e-02, -2.24418882e-02, ...,
         1.31472750e-02, -1.46121429e-02, -1.23816980e-02],
       [-1.16869673e-01,  1.47849992e-01, -1.29781947e-01, ...,
         5.06461710e-02, -6.30727336e-02, -7.84740597e-02],
       [-4.22876887e-02,  5.47618084e-02, -5.13737388e-02, ...,
         1.60699151e-02, -1.90709531e-02, -2.65847333e-02],
       ...,
       [ 4.38447595e-02,  8.31422135e-02,  1.54286285e-03, ...,
         2.70899385e-02, -6.35744305e-04, -4.96665090e-02],
       [-1.04750372e-01, -9.54229012e-02,  4.57641254e-05, ...,
         1.06209286e-01, -1.25129357e-01, -1.39116243e-01],
       [ 5.57465792e-01,  5.91069460e-01, -1.82163634e-03, ...,
         5.48851043e-02, -5.53776920e-02, -6.24051280e-02]], dtype=float32)

## Save features & labels

In [26]:
archive = dir_archive('hsd/Reddit/X_y_{}'.format(MODEL), {'features': features, 'labels': labels,
                                                                'wt_num': np.array(wordtoken_features).shape[1]}, serialized=True)
archive.dump()
del archive